## GP Regression on Edges of a Simplicial Complex


In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.append('..') 
import numpy as np
from utils.preprocessing import load_dataset
import torch
import gpytorch
from gpytorch.constraints import Positive
from kernels.edge_kernel_ocean_flow import DiffusionKernelOceanFlow, MaternKernelOceanFlow, DiffusionKernelOceanFlowNonHC, MaternKernelOceanFlowNonHC, LaplacianKernelOceanFlow, LaplacianKernelOceanFlowNonHC

from tqdm.notebook import tqdm 


### Load and preprocess the Dataset

In [7]:

data_normalization = False 

kernel_name = ['diffusion','matern','diffusion-nonhc','matern-nonhc','laplacian','laplacian-nonhc']

kernel_name = 'diffusion'
seed = 5

incidence_matrices, laplacians, data_train, data_test, data, eigenpairs = load_dataset(data_name='ocean_flow', train_ratio=0.2, seed=seed)
eigvecs, eigvals = eigenpairs
L1, L1_down, L1_up = laplacians
B1, B2 = incidence_matrices


In [8]:
total_var = []
total_div = []
total_curl = []
num_eigemodes = len(eigvals)
for i in range(num_eigemodes):
    total_var.append(eigvecs[:, i].T@L1@eigvecs[:, i])
    total_div.append(eigvecs[:, i].T@L1_down@eigvecs[:, i])
    total_curl.append(eigvecs[:, i].T@L1_up@eigvecs[:, i])
    
grad_eflow = np.where(np.array(total_div) >= 1e-4)[0]
curl_eflow = np.where(np.array(total_curl) >= 1e-3)[0]

In [10]:
# tensor data type 
data_type = torch.float32
# convert the laplacians first to a list then to torch sparse csr tensors
L1 = torch.sparse_coo_tensor(L1.nonzero(), L1.data, L1.shape)
L1_down = torch.sparse_coo_tensor(L1_down.nonzero(), L1_down.data, L1_down.shape)
L1_up = torch.sparse_coo_tensor(L1_up.nonzero(), L1_up.data, L1_up.shape)
grad_evectors = torch.tensor(eigvecs[:, grad_eflow], dtype=data_type)
curl_evectors = torch.tensor(eigvecs[:, curl_eflow], dtype=data_type)
grad_evalues = torch.tensor(eigvals[grad_eflow], dtype=data_type)
curl_evalues = torch.tensor(eigvals[curl_eflow], dtype=data_type)
laplacians = [L1, L1_down, L1_up]

/tmp/ipykernel_2036278/742530486.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:269.)
  L1 = torch.sparse_coo_tensor(L1.nonzero(), L1.data, L1.shape)


## Data Perprocessing

In [11]:
x_train, y_train = torch.tensor(data_train[0], dtype=data_type) , torch.tensor(data_train[1], dtype=data_type) 
x_test, y_test = torch.tensor(data_test[0], dtype=data_type) , torch.tensor(data_test[1], dtype=data_type)
x, y = torch.tensor(data[0], dtype=data_type), torch.tensor(data[1], dtype=data_type )

orig_mean, orig_std = torch.mean(y_train), torch.std(y_train)

if data_normalization:
    y_train = (y_train-orig_mean)/orig_std
    y_test = (y_test-orig_mean)/orig_std
    y = (y-orig_mean)/orig_std

output_device = torch.device('cuda:1')
x_train, y_train = x_train.to(output_device), y_train.to(output_device)
x_test, y_test = x_test.to(output_device), y_test.to(output_device)
x, y = x.to(output_device), y.to(output_device)


Move to GPU

In [12]:
# send to device
laplacians = [laplacian.to(output_device) for laplacian in laplacians]
# convert the eigenpairs to torch tensors
grad_evectors = grad_evectors.to(output_device)
curl_evectors = curl_evectors.to(output_device)
grad_evalues = grad_evalues.to(output_device)
curl_evalues = curl_evalues.to(output_device)
eigenpairs = [grad_evectors, curl_evectors, grad_evalues, curl_evalues]


In [13]:
torch.cuda.empty_cache()

# Edge Kernel Regression

In [14]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel, mean_function=None):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        if mean_function is None:
            self.mean_module = gpytorch.means.ConstantMean()
        elif mean_function == 'zero':
            self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel, outputscale_constraint=Positive())
        # self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

if kernel_name == 'diffusion':
    kernel = DiffusionKernelOceanFlow(eigenpairs)
elif kernel_name == 'diffusion-nonhc':
    kernel = DiffusionKernelOceanFlowNonHC(eigenpairs)
elif kernel_name == 'matern':
    kernel = MaternKernelOceanFlow(eigenpairs)
elif kernel_name == 'matern-nonhc':
    kernel = MaternKernelOceanFlowNonHC(eigenpairs)
elif kernel_name == 'laplacian':
    kernel = LaplacianKernelOceanFlow(eigenpairs)
elif kernel_name == 'laplacian-nonhc':
    kernel = LaplacianKernelOceanFlowNonHC(eigenpairs)
elif kernel_name == 'naive-matern':
    kernel = gpytorch.kernels.MaternKernel()
elif kernel_name == 'naive-rbf':
    kernel = gpytorch.kernels.RBFKernel()
    
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood, kernel, mean_function=None)

### Using the GPU

In [15]:
if torch.cuda.is_available():
   model = model.to(output_device)
   likelihood = likelihood.to(output_device)

In [16]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:50} value = {param.item()}')

Parameter name: likelihood.noise_covar.raw_noise                   value = 0.0
Parameter name: mean_module.raw_constant                           value = 0.0
Parameter name: covar_module.raw_outputscale                       value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_down            value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_up              value = 0.0
Parameter name: covar_module.base_kernel.raw_h_down                value = 0.0
Parameter name: covar_module.base_kernel.raw_h_up                  value = 0.0


#### Initialize the hyperprameters

In [17]:
# hypers = {
#     'likelihood.noise_covar.noise': torch.tensor(1.),
#     'covar_module.base_kernel.kappa_down': torch.tensor(1),
#     'covar_module.base_kernel.kappa_up': torch.tensor(1),
#     # 'covar_module.base_kernel.mu': torch.tensor(1),
#     'covar_module.outputscale': torch.tensor(1.),
#     'mean_module.raw_constant': orig_mean,
# }

# model.initialize(**hypers)
# print(
#     model.likelihood.noise_covar.noise.item(),
#     model.covar_module.base_kernel.kappa_down.item(),
#     model.covar_module.base_kernel.kappa_up.item(),
#     # model.covar_module.base_kernel.mu.item(),
#     model.covar_module.outputscale.item(),
#     model.mean_module.constant.item()
# )

### Train the GPR model 

In [18]:
training_iter = 200

In [19]:
# Wrap training, prediction and plotting from the ExactGP-Tutorial into a function,
# so that we do not have to repeat the code later on
def train(model, likelihood, training_iter=training_iter):
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    iterator = tqdm(range(training_iter))
    for i in iterator:
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(x_train)
        # Calc loss and backprop gradients
        loss = -mll(output, y_train)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (
                i + 1, training_iter, loss.item(),
        ))
        optimizer.step()

#### Train the model the analyze the results

In [20]:
model.train()
likelihood.train()
train(model, likelihood)

  0%|          | 0/200 [00:00<?, ?it/s]

Iter 1/200 - Loss: 1.383
Iter 2/200 - Loss: 1.373
Iter 3/200 - Loss: 1.361
Iter 4/200 - Loss: 1.353
Iter 5/200 - Loss: 1.345
Iter 6/200 - Loss: 1.334
Iter 7/200 - Loss: 1.320
Iter 8/200 - Loss: 1.305
Iter 9/200 - Loss: 1.288
Iter 10/200 - Loss: 1.268
Iter 11/200 - Loss: 1.244
Iter 12/200 - Loss: 1.218
Iter 13/200 - Loss: 1.189
Iter 14/200 - Loss: 1.159
Iter 15/200 - Loss: 1.126
Iter 16/200 - Loss: 1.088
Iter 17/200 - Loss: 1.049
Iter 18/200 - Loss: 1.009
Iter 19/200 - Loss: 0.968
Iter 20/200 - Loss: 0.927
Iter 21/200 - Loss: 0.886
Iter 22/200 - Loss: 0.842
Iter 23/200 - Loss: 0.802
Iter 24/200 - Loss: 0.767
Iter 25/200 - Loss: 0.728
Iter 26/200 - Loss: 0.696
Iter 27/200 - Loss: 0.666
Iter 28/200 - Loss: 0.639
Iter 29/200 - Loss: 0.620
Iter 30/200 - Loss: 0.605
Iter 31/200 - Loss: 0.598
Iter 32/200 - Loss: 0.591
Iter 33/200 - Loss: 0.582
Iter 34/200 - Loss: 0.588
Iter 35/200 - Loss: 0.586
Iter 36/200 - Loss: 0.578
Iter 37/200 - Loss: 0.584
Iter 38/200 - Loss: 0.592
Iter 39/200 - Loss: 0

In [21]:
torch.cuda.empty_cache()

### Predict


In [22]:
def predict(model, likelihood, x_test):
    model.eval()
    likelihood.eval()
    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        # Test points are regularly spaced along [0,1]
        return likelihood(model(x_test))

In [23]:
observed_pred = predict(model, likelihood, x_test)
pred_mean, pred_var = observed_pred.mean, observed_pred.variance
lower, upper = observed_pred.confidence_region()

In [25]:

print('Test MAE: {}'.format(gpytorch.metrics.mean_absolute_error(observed_pred, y_test)), '\n'
    'Test MSE: {}'.format(gpytorch.metrics.mean_squared_error(observed_pred, y_test)), '\n'
    'Test NLPD: {}'.format(gpytorch.metrics.negative_log_predictive_density(observed_pred, y_test))
)

Test MAE: 0.2513890266418457 
Test MSE: 0.11903209984302521 
Test NLPD: 0.3344270586967468


check model parameters

In [26]:
model.state_dict()

OrderedDict([('likelihood.noise_covar.raw_noise',
              tensor([-2.2917], device='cuda:1')),
             ('likelihood.noise_covar.raw_noise_constraint.lower_bound',
              tensor(1.0000e-04, device='cuda:1')),
             ('likelihood.noise_covar.raw_noise_constraint.upper_bound',
              tensor(inf, device='cuda:1')),
             ('mean_module.raw_constant', tensor(0.0077, device='cuda:1')),
             ('covar_module.raw_outputscale', tensor(7.5567, device='cuda:1')),
             ('covar_module.base_kernel.raw_kappa_down',
              tensor([[8.1434]], device='cuda:1')),
             ('covar_module.base_kernel.raw_kappa_up',
              tensor([[7.2782]], device='cuda:1')),
             ('covar_module.base_kernel.raw_h_down',
              tensor([[7.2791]], device='cuda:1')),
             ('covar_module.base_kernel.raw_h_up',
              tensor([[7.6236]], device='cuda:1')),
             ('covar_module.base_kernel.raw_kappa_down_constraint.lower_boun